# to build a representation of QM9 that contains only the stuff I need and will not crash my kernel.

In [ ]:
import math
from itertools import product
from typing import List, Any, Set
import random
import pandas as pd
import torch
from torch import Tensor, LongTensor
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [18]:
new_node_position_list = [0,
                71228,
                157492,
                243324,
                326352,
                402375,
                483859,
                569011,
                663266,
                745680,
                828680,
                917385,
                1018451,
                1111755,
                1204441,
                1297886,
                1391805,
                1487137,
                1585742,
                1683406,
                1783800,
                1883738,
                1983325,
                2085970,
                2190791,
                2272377,
                2333625
                ]

new_graph_position_list = list(range(0,125000,5000)) + [129433]

In [21]:
path_head = 'QM9/'
with open('Zebra/QM9.csv', 'a') as file:
    for i in range(len(new_node_position_list)-1):
        sn = new_node_position_list[i]
        en = new_node_position_list[i+1]
        
        sg = new_graph_position_list[i]
        eg = new_graph_position_list[i+1]
        
        node_features_df = pd.read_csv(path_head + 'node_attributes.txt', skiprows=sn, header=None, nrows=en-sn)
        node_features = torch.tensor(node_features_df.values)

        graph_features_df = pd.read_csv(path_head + 'Y.txt', skiprows=sg, header=None, nrows=eg-sg)
        graph_features = torch.tensor(graph_features_df.values)

        atomic_numbers = node_features[:,5].long()

        positions = node_features[:,-3:]

        internal_energies = graph_features[:,7]
        internal_energies_normalized = torch.nn.functional.normalize(internal_energies, dim=0)
        internal_energies_normalized_list = internal_energies_normalized.tolist()

        new_graph_node_indices = [0]
        node_indicators_df = pd.read_csv(path_head + 'graph_indicator.txt', skiprows=sn, header=None, nrows=en-sn)
        node_indicators = node_indicators_df.values.tolist()
        for ix in range(len((node_indicators)))[1:]:
            if node_indicators[ix] != node_indicators[ix-1]:
                new_graph_node_indices.append(ix)
        new_graph_node_indices.append(len(node_indicators))

        for start, end in zip(new_graph_node_indices, new_graph_node_indices[1:]):
            x = atomic_numbers.clone()[start:end].view(-1, 1)
            y = internal_energies_normalized_list.pop(0)
            pos = positions.clone()[start:end]
            string_builder = '['
            for a in x:
                string_builder+=str(a.item())+','
            string_builder = f'{string_builder[:-1]}]['
            for p in pos:
                string_builder += f'[{str(p[0].item())},{str(p[1].item())},{str(p[2].item())}],'
            string_builder = f'{string_builder[:-1]}]'
            string_builder += f'[{y}]\n'
            file.write(string_builder)
        
    file.close()